In [2]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import wandb
wandb.login()
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# set seed for reproducibility
random_seed = 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: grantbell. Use `wandb login --relogin` to force relogin


In [3]:
df = pd.read_csv('fred_230718.csv', index_col='Date', parse_dates=True)
df = df.asfreq('QS')
earliest_date = '1963-01-01'
latest_date = '2021-10-01'
# # filter df index to be between earliest_date and latest_date
df = df.loc[(df.index >= earliest_date) & (df.index <= latest_date)]
df.dropna(axis=1, inplace=True)
df.head()

,GDP,GDPC1,GDPPOT,CPIAUCSL,CPILFESL,GDPDEF,M1V,M2V,DFF,UNRATE,...,MANEMP,DSPIC96,PCE,PCEDG,PSAVERT,DSPI,INDPRO,HOUST,GPDI,MSPUS
Date,,,,,,,,,,,,,,,,,,,,,
1963-01-01,621.672,3628.306,3662.738125,30.44,31.5,17.134,4.178,1.690,3.00,5.7,...,15545.0,2541.1,374.4,53.1,10.9,430.0,26.0448,1244.0,99.689,17800.0
1963-04-01,629.752,3669.020,3701.698767,30.48,31.7,17.164,4.194,1.675,3.00,5.7,...,15602.0,2547.1,376.4,53.2,10.7,431.1,26.7473,1689.0,101.650,18000.0
1963-07-01,644.444,3749.681,3741.388301,30.69,31.8,17.187,4.248,1.680,3.00,5.6,...,15646.0,2572.6,384.4,55.5,10.1,438.0,27.0445,1614.0,104.612,17900.0
1963-10-01,653.938,3774.264,3781.880559,30.75,32.0,17.326,4.269,1.672,3.50,5.5,...,15714.0,2617.3,386.0,54.2,11.5,447.0,27.5578,1779.0,107.189,18500.0
1964-01-01,669.822,3853.835,3822.450115,30.94,32.2,17.381,4.345,1.685,3.25,5.6,...,15715.0,2652.8,396.8,57.9,10.7,455.3,27.8820,1603.0,110.474,18500.0


set target and create, train, validate, and test datasets and then scale and transform them so they will work better with the neural network

In [4]:
target = 'MSPUS'
y = df[target]
X = df.drop(columns=[target]).shift(1).dropna()
y = y.loc[X.index] # Make sure y and X have the same rows after dropna

# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=random_seed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # validation data should also be scaled
X_test_scaled = scaler.transform(X_test)

# Log-transform the target variable
y_train_log = np.log1p(y_train)
y_valid_log = np.log1p(y_valid)

In [5]:
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

def train_model(X_train, y_train, 
                X_valid, y_valid,
                layer_sizes=[100, 100], 
                activation="relu", 
                kernel_initializer="he_normal", 
                loss='mse',
                learning_rate=0.001, 
                epochs=100,
                batch_norm=False,
                l1_l2=False,
                l1=.01,
                l2=.01,
                metrics=['mse']):

    # Create a sequential model
    model = tf.keras.Sequential()

    # Add batch normalization and dense layers according to the layer_sizes
    for size in layer_sizes:
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        if l1_l2:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer
                                            , kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        else:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer))

    # Add a final Dense layer with no activation
    model.add(tf.keras.layers.Dense(1))

    # Create the optimizer with the custom learning rate
    sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(loss=loss, optimizer=sgd, metrics=metrics)
    
    # Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints
    wandb_callbacks = [
        WandbMetricsLogger(),
        # WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
        # WandbModelCheckpoint(filepath="my_model_best", save_best_only=True, monitor='val_loss'),
    ]

    # Train the model using the scaled data
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=wandb_callbacks)
    
    return model

In [6]:
sweep_config = {
    'method': 'bayes',  # can be grid, random, or bayes
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.01
        },
        'activation': {
            'values': ['relu']
        }, 
        'kernel_initializer': {
            'values': ['he_normal']
        }, 
        'loss': {
            'values': ['mse']
        }, 
        # 'activation': {
        #     'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu', 'softplus']  # more activation functions
        # }, 
        # 'kernel_initializer': {
        #     'values': ['he_normal', 'glorot_uniform', 'glorot_normal', 'lecun_normal']  # more initializers
        # }, 
        # 'loss': {
        #     'values': ['mse', 'mae', 'logcosh', 'huber']  # more loss functions
        # }, 
        'epoch': {
            'min': 10,
            'max': 500
        },
        'batch_norm': {
            'values': [True, False]
        }, 
        'l1_l2': {
            'values': [True]
        },        
        'l1': {
            'min': 0.0001,
            'max': 0.5
        },
        'l2': {
            'min': 0.0001,
            'max': 0.5
        },
        'metrics': {
            'values': ['mae']
        }
        # 'metrics': {
        #     'values': ['mae', 'mse', 'mape', 'msle']  # adding more metrics
        # }
    }
}

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = train_model(
            X_train=X_train_scaled, y_train=y_train_log,
            X_valid=X_test_scaled, y_valid=y_valid_log,
            activation=config.activation, kernel_initializer=config.kernel_initializer, 
            loss=config.loss, learning_rate=config.learning_rate, 
            epochs=config.epoch, batch_norm=config.batch_norm, 
            l1_l2=config.l1_l2, l1=config.l1, l2=config.l2, 
            metrics=config.metrics)

sweep_id = wandb.sweep(sweep_config, project="fred_dnn_sweep")
# wandb.agent(sweep_id, train, count=100)
wandb.agent(sweep_id, train, count=10)

Create sweep with ID: texsheds
Sweep URL: https://wandb.ai/grantbell/fred_dnn_sweep/sweeps/texsheds


wandb: Agent Starting Run: avhlos0q with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	epoch: 490
wandb: 	kernel_initializer: he_normal
wandb: 	l1: 0.26001267383913895
wandb: 	l1_l2: True
wandb: 	l2: 0.12403897658461828
wandb: 	learning_rate: 0.009927344296418127
wandb: 	loss: mse
wandb: 	metrics: mae
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/490
5/5 [==============================] - 1s 48ms/step - loss: 570.3391 - mae: 8.7908 - val_loss: 469.9981 - val_mae: 3.8360
Epoch 2/490
5/5 [==============================] - 0s 10ms/step - loss: 442.3130 - mae: 2.5543 - val_loss: 407.1049 - val_mae: 2.2991
Epoch 3/490
5/5 [==============================] - 0s 9ms/step - loss: 388.3901 - mae: 1.4074 - val_loss: 360.2989 - val_mae: 1.7185
Epoch 4/490
5/5 [==============================] - 0s 9ms/step - loss: 343.6057 - mae: 1.0487 - val_loss: 320.3231 - val_mae: 1.5308
Epoch 5/490
5/5 [==============================] - 0s 11ms/step - loss: 308.1617 - mae: 2.0513 - val_loss: 296.2966 - val_mae: 3.9805
Epoch 6/490
5/5 [==============================] - 0s 10ms/step - loss: 275.4482 - mae: 2.6122 - val_loss: 252.0265 - val_mae: 1.7217
Epoch 7/490
5/5 [==============================] - 0s 9ms/step - loss: 238.8707 - mae: 1.7510 - val_loss: 220.4049 - val_mae: 2.1397
Epoch 8/490
5/5 [==============================] - 0s 10ms/step -

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,▆█▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_mae,█▇▅▁▂▁▁▂▂▁▂▂▁▂▂▂▁▂▁▂▂▂▂▂▂▂▁▂▂▂▂▁▂▂▁▂▂▂▂▂
epoch/epoch,489
epoch/learning_rate,0.00993
epoch/loss,4.5063
epoch/mae,0.11592
epoch/val_loss,5.98843


wandb: Agent Starting Run: 4yftrasj with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	epoch: 484
wandb: 	kernel_initializer: he_normal
wandb: 	l1: 0.16858421143402982
wandb: 	l1_l2: True
wandb: 	l2: 0.2045356052927331
wandb: 	learning_rate: 0.0010338635605050405
wandb: 	loss: mse
wandb: 	metrics: mae
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

Epoch 1/392
5/5 [==============================] - 1s 53ms/step - loss: 186.4918 - mae: 4.4206 - val_loss: 154.6499 - val_mae: 3.6466
Epoch 2/392
5/5 [==============================] - 0s 13ms/step - loss: 140.3049 - mae: 2.1173 - val_loss: 156.2762 - val_mae: 4.6788
Epoch 3/392
5/5 [==============================] - 0s 12ms/step - loss: 129.1065 - mae: 1.4226 - val_loss: 154.5132 - val_mae: 5.0845
Epoch 4/392
5/5 [==============================] - 0s 12ms/step - loss: 122.9298 - mae: 1.4882 - val_loss: 144.4313 - val_mae: 4.9880
Epoch 5/392
5/5 [==============================] - 0s 12ms/step - loss: 114.8048 - mae: 1.3163 - val_loss: 138.6186 - val_mae: 5.1523
Epoch 6/392
5/5 [==============================] - 0s 11ms/step - loss: 107.0694 - mae: 0.7151 - val_loss: 138.4131 - val_mae: 5.8096
Epoch 7/392
5/5 [==============================] - 0s 12ms/step - loss: 101.2252 - mae: 0.8134 - val_loss: 130.9526 - val_mae: 5.5996
Epoch 8/392
5/5 [==============================] - 0s 11ms/ste

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▂▂▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,▄▂▃▃▂▃▂▅▂▃▄▃▂▂█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▆▅▃▃▂▂▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_mae,▅▆▆▅▅▄▃▂▂▂▂▂▁▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
epoch/epoch,391
epoch/learning_rate,0.00961
epoch/loss,0.07545
epoch/mae,0.10536
epoch/val_loss,2.25178


wandb: Agent Starting Run: qtcthvo0 with config:
wandb: 	activation: relu
wandb: 	batch_norm: False
wandb: 	epoch: 193
wandb: 	kernel_initializer: he_normal
wandb: 	l1: 0.4936476396644862
wandb: 	l1_l2: True
wandb: 	l2: 0.1012483242517488
wandb: 	learning_rate: 0.006587154491989599
wandb: 	loss: mse
wandb: 	metrics: mae
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/193
5/5 [==============================] - 0s 38ms/step - loss: 905.6235 - mae: 5.7412 - val_loss: 798.1454 - val_mae: 2.4191
Epoch 2/193
5/5 [==============================] - 0s 10ms/step - loss: 760.0655 - mae: 1.7069 - val_loss: 697.4252 - val_mae: 1.6602
Epoch 3/193
5/5 [==============================] - 0s 10ms/step - loss: 664.8269 - mae: 1.4101 - val_loss: 612.9308 - val_mae: 2.4940
Epoch 4/193
5/5 [==============================] - 0s 10ms/step - loss: 579.5765 - mae: 1.3725 - val_loss: 529.9196 - val_mae: 1.8092
Epoch 5/193
5/5 [==============================] - 0s 10ms/step - loss: 501.5380 - mae: 0.9384 - val_loss: 458.7227 - val_mae: 1.8213
Epoch 6/193
5/5 [==============================] - 0s 10ms/step - loss: 434.2468 - mae: 1.2304 - val_loss: 395.8134 - val_mae: 1.4041
Epoch 7/193
5/5 [==============================] - 0s 10ms/step - loss: 374.4805 - mae: 1.3488 - val_loss: 345.0327 - val_mae: 2.7943
Epoch 8/193
5/5 [==============================] - 0s 10ms/ste

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,█▆▄▄▃▁▁▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_mae,▇▅▄█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch/epoch,192
epoch/learning_rate,0.00659
epoch/loss,10.63251
epoch/mae,0.20297
epoch/val_loss,11.93227
